# Topic modelling using BERTopic

In [5]:
import utilities
from bertopic import BERTopic
from sklearn.feature_extraction.text import CountVectorizer
import nltk
nltk.download('stopwords')
from hdbscan import HDBSCAN
import pandas as pd
import pickle

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\a.tekiouk\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
corpus_chan_10k = pd.read_csv("C:/Users/a.tekiouk/Sujet_2/Sujet_2/DATA/chanel10k.csv", sep=";", parse_dates=["publication_time"])

# on récupére les posts en anglais
mask = corpus_chan_10k["language"] == 'en'
corpus_chan_10k = corpus_chan_10k.loc[mask]

In [6]:
with open('C:/Users/a.tekiouk/Sujet_2/Sujet_2/Models/model_auc.pkl', 'rb') as f:
    model_f = pickle.load(f) # deserialize using load()
with open('C:/Users/a.tekiouk/Sujet_2/Sujet_2/Models/opt_reducer.pkl', 'rb') as f:
    reducer_f = pickle.load(f) # deserialize using load()
corpus_filtered = utilities.junk_classifier(
    Corpus_instagram=corpus_chan_10k,
    model=model_f,
    reducer=reducer_f)

KeyboardInterrupt: 

In [ ]:
stop_words = stopwords.words('english')

vectorizer = CountVectorizer(stop_words=stop_words)
                                             
umap = UMAP(n_neighbors=15, n_components=5, min_dist=0.0, metric='cosine', random_state=42)
hdbscan = HDBSCAN(min_cluster_size=10, min_samples=1, metric='euclidean', cluster_selection_method='eom')
# representation_model = MaximalMarginalRelevance(diversity=0.30)

bert_topic_model = BERTopic(
    embedding_model=model,
    umap_model=umap, 
    hdbscan_model=hdbscan,
    vectorizer_model=vectorizer,
    # top_n_words=10,
    verbose=True,
    # representation_model = representation_model 
)

embeddings = encoder.encode(docs)


docs = corpus_filtered['text'].tolist()
bert_topic_model.fit(docs, embeddings)
print(f"Total topics found by BERTopic: {len(bert_topic_model.topic_labels_.keys())} (the topic -1 represents the outliers)")
#CountVectorizer ?
#stopwords ?

In [ ]:
bert_topic_model.get_topic_info()

In [ ]:
bert_topic_model.visualize_topics()

In [ ]:
bert_topic_model.visualize_heatmap()

In [ ]:
bert_topic_model.visualize_hierarchy()

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt

def create_wordcloud(topic_model, topic):
    text = {word: value for word, value in topic_model.get_topic(topic)}
    wc = WordCloud(background_color="white", max_words=1000)
    wc.generate_from_frequencies(text)
    plt.imshow(wc, interpolation="bilinear")
    plt.axis("off")
    plt.show()

create_wordcloud(bert_topic_model, topic=-1)